# Using Syringe Pumps and Motion Controllers

In this notebook, a motion controller and syringe pump controllers are used to create an array of drop-cast films.

*Pyvisa - python pacakge enables communication between python programs and electronic instruments.* <br>
* Instal pyvisa via: pip install -U pyvisa-py, in command prompt

In [ ]:
#Import Packages
from cgi import test
from optparse import Values
from sqlite3 import Row
from time import time
import tkinter as tk
from turtle import width
import pyvisa
import cv2
from PIL import Image, ImageTk
from webbrowser import get
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import threading as th
import minimalmodbus
from skopt import Optimizer
from skopt.plots import plot_objective
from skopt.plots import plot_evaluations
import os

Create a resource manager object, responsible for managing connections and communication with various instruments connected to your computer

In [ ]:
rm = pyvisa.ResourceManager()

In [ ]:
print(rm.list_resources()) 

Now establish a communication with a specific instrument connected to the port

In [ ]:
#ASRL7 = Linear Stage
com_pxpy = rm.open_resource('ASRL7::INSTR')
com_pxpy.baud_rate = 921600 #fixed

#ASRL4 = Syringe pumps
com_syr = rm.open_resource('ASRL4::INSTR')
com_syr.baud_rate = 115200 #fixed

# Calibrate the Motion Controller

Instruct motion controller to enable motion for axis 1 and axis 2. Identify the live position of motion controller and print it.

In [ ]:
#Turns on Motion Controller
com_pxpy.write('1MO')
com_pxpy.write('2MO')

#Identify current Motion Controller position and print
live_position_x_t = com_pxpy.query_ascii_values('1TP?')
live_position_x = ''.join(str(element) for element in live_position_x_t)
live_position_y_t = com_pxpy.query_ascii_values('2TP?')
live_position_y = ''.join(str(element) for element in live_position_y_t)
print('(x,y):')
print(live_position_x, live_position_y)


Now define a function to command motion controller to move to a specified position

In [ ]:
def move_to(position_x, position_y):
    position_x = str(position_x)
    position_y = str(position_y)
    com_pxpy.write('1PA'+position_x)
    time.sleep(0.1)              
    com_pxpy.write('2PA'+position_y)
    time.sleep(0.1)

Defining a position where syringe aligns to the flush position and check by identifying the syringe position

In [ ]:
move_to(-40,-40) #This will be Syringe Position

In [ ]:
#Identifying the Syringe position
syringe_position_x = com_pxpy.query_ascii_values('1TP?')
syringe_position_x = ''.join(str(element) for element in syringe_position_x)
syringe_position_y = com_pxpy.query_ascii_values('2TP?')
syringe_position_y = ''.join(str(element) for element in syringe_position_y)
print(syringe_position_x,syringe_position_y)

# Setup the Syringe Pumps for Experiments (here 2 pumps are used)

Initialize the syringe and drying parameters:

* Syringe Diameter (mm)

* Reservior Volume (mL)

* Total volume to be dispensed (uL)

* Dispense Time (t) 

* Infusion Rate (mL/min)

* Evaporation time (t) 

* Sample center-center distance (mm)

In [ ]:
#Initialize syringe parameters

sy_dia = 10.3 #mm (hamiliton glass syringes 81520)
res_vol = 4 #mL

total_dispense_vol = V_d = 16 #uL
dispense_time = t_d = 30 #s

Q_d = 60 * (V_d/t_d) #mL/min
evaporation_time = e_t = 30 #=  18000 #s

sample_distance = 20 #mm

Define a function to execute syring pumping:

In [ ]:
#Defining an operation that executes the pumping procedure
def single_pump(sy_dia, res_vol, ID, rate, dispense_vol):
    com_syr.write(str(ID) + 'svolume ' + str(res_vol) + ' ml')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'diameter ' + str(float(sy_dia)))
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irate ' + str(float(rate)) + ' ul/min')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'tvolume ' + str(float(dispense_vol)) + ' ul')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irun')

# Initial Flush Through Tubing System

Initial flushing fills the tubing with liquid. Volume for flushing is calculated based on:

* Inner diameter of tubing is 1/32”

* Radius = 1/64 Inch =  0.396875 mm

* Volume of tubing = pi* R ** 2* L = 0.495 * L (mm ** 3)

* Volume of piped = N (mm ** 3) = N * 0.001 mL =  0.495 * L *  0.001 mL

In [ ]:
L_pipes = 400 #mm
V_flush_mL = 0.495 * L_pipes * 0.001 #mL #1/16 system
V_flush_uL = V_flush_mL * 1000 # Convert to uL

V_flush_initial = V_flush_uL
print(V_flush_initial)

Now define a function "flush()" to flush the piping system.

In [ ]:
def flush(flush_volume_0, flush_volume_1, flush_time): #flushes syringe 1 & 2
    #minki made flush_time = 3*flush_volume (around 20s)
    
    #calculation of rate of flush
    rate_flush_0 = 60 * (flush_volume_0 / flush_time)
    rate_flush_1 = 60 * (flush_volume_1 / flush_time)
    
    #flush the piping
    single_pump(sy_dia, res_vol, 0, rate_flush_0, flush_volume_0)
    time.sleep(0.1)

    single_pump(sy_dia, res_vol, 1, rate_flush_1, flush_volume_1)
    time.sleep(0.1)

Excecute the initial flushing through pipes

In [ ]:
#Initial flsh through pipes
move_to(syringe_position_x,syringe_position_y)
time.sleep(0.2)
flush(V_flush_initial, V_flush_initial, 10)
time.sleep(0.2)

# Setup the Dropcast Loop

This is the core of dropcasting process, which includes depositing a droplet onto the substrate, waiting for evaporation, moving to the camera position for image analysis, and making any necessary optimizations or adjustments for the next set of parameters.

Droplet Volume and Flow rate is calculated based on this:

V = Q * t

	Volume to be dispensed = V_d
	Dispense time = t_d
	Dispense Rate = Q_d
	
	V_d = V_0 + V_1 

	V_d = Q_d *  t_d = (Q_0 + Q_1) * t_d


p = percentage of fluid one; (1-p) percentage of other fluid

	V_d = V_0 + V_1 = pV_d - (1-p)V_d
	V_0 = (p)V_d
	V_1 = (1-p)V_d 
    
"p" represents the concentration of the second syringe pump relative to the first, which is considered as an AE parameter

In [ ]:
#AE parameters 
file_path = "image_path"
#len of p is the number of droplets
p = [0.4,0.2]    

The dropcast loop iterates until all the droplets have been deposited

For each iteration:

1. The location of the "i-th" droplet  based on a predetermined pattern
2. The volume and infusion rate for the "i-th" is calculated based on predefined parameters 
3. The syringe is moved to the location of the "i-th" droplet and starts pumping the calculated volumes
4. Time for droplet evaporation is waited

Note: Flush setup can also be used in the loop to clear the piping for a fresh droplet, but in this instance, it is not necessary.

In [ ]:
# Dropcasting Loop
for i in range(len(p)):
    # Print current iteration
    print(f'iteration {i+1}')
    time.sleep(0.2)
    
    # Find location of each droplet in iteration
    n = 2  # number of droplets per row
    a, b = divmod(i, n)  # Calculate row and column indices
    print(a, b)
    
    #flush is to the right
    position_x = str(float(syringe_position_x)-(sample_distance*int(b)))
    position_y = str(float(syringe_position_y)-(sample_distance*int(a)))
    print(f'droplet center:{position_x,position_y}')
    
    # Calculate droplet volumes based on a predefined ratio
    V_0 = p[i] * V_d  # Volume for droplet 0
    V_1 = (1 - p[i]) * V_d  # Volume for droplet 1
    
    # Calculate infusion rates for each droplet
    Q_0 = 60 * (V_0 / t_d)  # Infusion rate for droplet 0
    Q_1 = 60 * (V_1 / t_d)  # Infusion rate for droplet 1
    
    print(f'droplet volume: {V_0, V_1}')
    
    # Move to the location of droplet i and start pumping
    move_to(position_x, position_y)
    single_pump(sy_dia, res_vol, 0, Q_0, V_0)  # Pump for droplet 0
    time.sleep(0.2)
    single_pump(sy_dia, res_vol, 1, Q_1, V_1)  # Pump for droplet 1
    time.sleep(0.2)
    
    # Allow droplet evaporation
    time.sleep(e_t)
    time.sleep(20)
    
    # End of loop
    print('end of loop')